# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression 

from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.dummy import DummyClassifier

df = pd.read_csv('/datasets/users_behavior.csv')

display(df.describe())

print('Количество дубликатов:', df.duplicated().sum())
print()
print(df.info())

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


Количество дубликатов: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


## Разбейте данные на выборки

Разобьем данные в соотношении 60% - 20% - 20%, для обучающего, валидационного и тестового датасета.

In [2]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

features_train, features_tmp, target_train, target_tmp = train_test_split(features, target, train_size=0.6, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_tmp, target_tmp, train_size=0.5, random_state=12345)

In [3]:
# Код ревьюера
features_train.shape, features_test.shape, features_valid.shape

((1928, 4), (643, 4), (643, 4))

## Исследуйте модели

Рассмотрим сначала алгоритм "Решающего Древа". Проведем анализ для разных глубин алгоритма.

In [4]:
best_result = 0.7
best_depth = 0
for depth in range(1,5):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split = 10)
    model_tree.fit(features_train, target_train)
    predictions_valid = model_tree.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
            best_model_tree = model_tree
            best_result = result
            best_depth = depth
            
    
print("max_depth =", best_depth, ": ", best_result)
    


max_depth = 3 :  0.7853810264385692


Наиболее эффективным показало себя древо с 3 уровнем глубины, дальше заметно переобучение модели.

Дальше рассмотрим модель "Случайного леса" с различными параметрами алгоритма.

In [5]:
best_model = None
best_result = 10000
best_est = 0
best_depth = 0
for est in range(10, 101, 10):
    for depth in range (1, 5):
        model_forest = RandomForestRegressor(random_state=12345, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        result = model_forest.score(features_valid, target_valid)
        if result < best_result:
            best_model_forest = model_forest
            best_result = result
            best_est = est
            best_depth = depth

print("Accuracy наилучшей модели на валидационной выборке:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели на валидационной выборке: 0.1140729465113487 Количество деревьев: 80 Максимальная глубина: 4


Модель слишком сложна для такого малого количества данных, поэтому accuracy низок.

Далее расссмотрим алгоритм линейной регрессии.

In [6]:
model_regression = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model_regression.fit(features, target)
result = model_regression.score(features_valid, target_valid)

print("Accuracy модели на валидационной выборке:", result)

Accuracy модели на валидационной выборке: 0.7573872472783826


**Выводы**

Наиболее точной моделью оказалась модель "Решающего Дерева", далее будем использовать именно её.

## Проверьте модель на тестовой выборке

Проверим модель "Решающего Дерева" на тестовой выборке и посмотрим на результаты accuracy.

In [7]:
predictions_test = best_model_tree.predict(features_test)
result = accuracy_score(target_test, predictions_test)
print('Accuracy модели на тестовой выборке:', result)

Accuracy модели на тестовой выборке: 0.7791601866251944


## Выводы

По полученным результатам можно утверждать, что для выбора тарифного плана наиболее подходящего клиенту подходит модель "Решающего Древа" с accuracy - 0,78 на тестовой выборке.

Применение предложенной модели в 80% случаев правильно предлагает клиентам тариф.

## (бонус) Проверьте модели на адекватность

In [8]:
dummy_cl = DummyClassifier(strategy="most_frequent")
dummy_cl.fit(features_train, target_train)
result = dummy_cl.score(features_valid, target_valid)
print('Accuracy простой модели DummyClassifier:', result)

Accuracy простой модели DummyClassifier: 0.7060653188180405


Accuracy модели DummyClassifier выше чем у модели RandomForestClassifier, поэтому можно утверждать что модель RandomForestClassifier неадекватна.